In [1]:
import os, shutil
import io
from io import StringIO
import re
import sys
from contextlib import redirect_stdout
from keras import models
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from keras import backend as K

import time

import numpy as np
import scipy.optimize as opt
import scipy.stats as sp
from scipy.stats import kde
import importlib as im
from sklearn import metrics
import csv

# my utilities
import cnn_utilities as cn
import uq_utilities_2 as uq

Using TensorFlow backend.


ERROR! Session/line number was not unique in database. History logging moved to new session 3722


In [3]:
## define pinball loss functions
qq = 0.75
def pinball_loss(y_true, y_pred, tau):
    err = y_true - y_pred
    return K.mean(K.maximum(tau*err, (tau-1)*err), axis=-1)

def pinball_loss_lower(y_true, y_pred):
    return pinball_loss(y_true, y_pred, tau = (1-qq)/2)

def pinball_loss_upper(y_true, y_pred):
    return pinball_loss(y_true, y_pred, tau = 1 - (1-qq)/2)

ERROR! Session/line number was not unique in database. History logging moved to new session 3721


In [18]:
# LOAD trained models and normalization values
# point est model
point_est_model = models.load_model("../saved_models/train_extant_R0_sampleRate_migrationRate.hdf5")

mean_sd = pd.read_csv("trained_quantile_CNN/cqr_train_extant_normalization_label_mean_sd.csv",
                           index_col=0).to_numpy()


# quantile models
q95_model = models.load_model("trained_quantile_CNN/cqr95_train_extant_R0_sampleRate_migrationRate.hdf5",
                              custom_objects = {'pinball_loss_lower': pinball_loss_lower, 
                                                'pinball_loss_upper': pinball_loss_upper})

q90_model = models.load_model("trained_quantile_CNN/cqr90_train_extant_R0_sampleRate_migrationRate.hdf5",
                              custom_objects = {'pinball_loss_lower': pinball_loss_lower, 
                                                'pinball_loss_upper': pinball_loss_upper})


q75_model = models.load_model("trained_quantile_CNN/cqr75_train_extant_R0_sampleRate_migrationRate.hdf5",
                              custom_objects = {'pinball_loss_lower': pinball_loss_lower, 
                                                'pinball_loss_upper': pinball_loss_upper})

q50_model = models.load_model("trained_quantile_CNN/cqr50_train_extant_R0_sampleRate_migrationRate.hdf5",
                              custom_objects = {'pinball_loss_lower': pinball_loss_lower, 
                                                'pinball_loss_upper': pinball_loss_upper})


q25_model = models.load_model("trained_quantile_CNN/cqr25_train_extant_R0_sampleRate_migrationRate.hdf5",
                              custom_objects = {'pinball_loss_lower': pinball_loss_lower, 
                                                'pinball_loss_upper': pinball_loss_upper})

q10_model = models.load_model("trained_quantile_CNN/cqr10_train_extant_R0_sampleRate_migrationRate.hdf5",
                              custom_objects = {'pinball_loss_lower': pinball_loss_lower, 
                                                'pinball_loss_upper': pinball_loss_upper})


q05_model = models.load_model("trained_quantile_CNN/cqr05_train_extant_R0_sampleRate_migrationRate.hdf5",
                              custom_objects = {'pinball_loss_lower': pinball_loss_lower, 
                                                'pinball_loss_upper': pinball_loss_upper})






In [6]:
# load trained model and normalization values
train_means = mean_sd[0,:]
train_sd = mean_sd[1,:]
train_aux_priors_means = train_means[3:,]
train_aux_priors_sd = train_sd[3:,]

num_locs = 5
max_tips = 502


In [7]:

############ checking coverage #################


In [19]:
# calibration data set
uq_cal_data = pd.read_csv("data_files/labels_and_preds/uq_calibration_sets_0to40.cblv.csv",
                            header =None, error_bad_lines = False, index_col = 0).to_numpy()
uq_cal_labels = pd.read_table("data_files/labels_and_preds/uq_calibration_sets_0to40_labels.tsv", header = 0).to_numpy()
uq_cal_mutips = pd.read_table("data_files/labels_and_preds/uq_calibration_sets_0to40_numtips_propsamp.txt", header = 0).to_numpy()
uq_cal_mutips = np.column_stack((uq_cal_mutips[:,0], uq_cal_mutips[:,1]/uq_cal_mutips[:,2]))

In [ ]:
# split calibration prediction dat for conformal prediction interval estimation and validation
uq_num_val = 5000

# for measuring resulting coverage
cal_val_preds = cnn_uq_cal_log_preds[0:uq_num_val,:]
cal_val_labels = uq_cal_labels[0:uq_num_val,:]

# for creating UQ
cal_preds = cnn_uq_cal_log_preds[uq_num_val:,:]
cal_labels = uq_cal_labels[uq_num_val:,:]

In [20]:
# create input tensors
uq_cal_subsample_prop = uq_cal_data[:,(max_tips-1) * 7]
uq_cal_mu = uq_cal_data[:,(max_tips - 3) * 7]
uq_cal_num_tips = cn.get_num_tips(uq_cal_data)

aux_uq_cal = np.vstack((uq_cal_mu, uq_cal_subsample_prop, uq_cal_num_tips,
                          uq_cal_labels[:,8], uq_cal_labels[:,9])).transpose()

norm_aux_uq_cal = cn.normalize(aux_uq_cal, (train_aux_priors_means, train_aux_priors_sd))

# create input tensors
aux_uq_cal_treeLocation_tensor, aux_uq_cal_prior_tensor = cn.create_data_tensors(data = uq_cal_data, 
                                                                                    mu = norm_aux_uq_cal[:,0],
                                                                                    subsample_prop = norm_aux_uq_cal[:,1],
                                                                                    num_tips = norm_aux_uq_cal[:,2],
                                                                                    tmrca = norm_aux_uq_cal[:,3],
                                                                                    mean_bl = norm_aux_uq_cal[:,4],
                                                                                    num_locs = num_locs,
                                                                                    max_tips = max_tips,
                                                                                    cblv_contains_mu_rho = True)

In [37]:
# PREDICT R0, sample rate, migration rate
uq95_normalized_preds = q95_model.predict([aux_uq_cal_treeLocation_tensor, aux_uq_cal_prior_tensor])
uq90_normalized_preds = q90_model.predict([aux_uq_cal_treeLocation_tensor, aux_uq_cal_prior_tensor])
uq75_normalized_preds = q75_model.predict([aux_uq_cal_treeLocation_tensor, aux_uq_cal_prior_tensor])
uq50_normalized_preds = q50_model.predict([aux_uq_cal_treeLocation_tensor, aux_uq_cal_prior_tensor])
uq25_normalized_preds = q25_model.predict([aux_uq_cal_treeLocation_tensor, aux_uq_cal_prior_tensor])
uq10_normalized_preds = q10_model.predict([aux_uq_cal_treeLocation_tensor, aux_uq_cal_prior_tensor])
uq05_normalized_preds = q05_model.predict([aux_uq_cal_treeLocation_tensor, aux_uq_cal_prior_tensor])

# reversing normalization
cnn_uq95_preds = np.exp(cn.denormalize(uq95_normalized_preds, train_means[0:3], train_sd[0:3]))
cnn_uq90_preds = np.exp(cn.denormalize(uq90_normalized_preds, train_means[0:3], train_sd[0:3]))
cnn_uq75_preds = np.exp(cn.denormalize(uq75_normalized_preds, train_means[0:3], train_sd[0:3]))
cnn_uq50_preds = np.exp(cn.denormalize(uq50_normalized_preds, train_means[0:3], train_sd[0:3]))
cnn_uq25_preds = np.exp(cn.denormalize(uq25_normalized_preds, train_means[0:3], train_sd[0:3]))
cnn_uq10_preds = np.exp(cn.denormalize(uq10_normalized_preds, train_means[0:3], train_sd[0:3]))
cnn_uq05_preds = np.exp(cn.denormalize(uq05_normalized_preds, train_means[0:3], train_sd[0:3]))


ERROR! Session/line number was not unique in database. History logging moved to new session 3733


In [195]:
# holdout a validation set for after calibration
# split calibration prediction dat for conformal prediction interval estimation and validation
uq_num_val = 5000

cal_uq_labels = uq_cal_labels[uq_num_val:,:]
cal_uq95_preds = cnn_uq95_preds[:,uq_num_val:,:]
cal_uq90_preds = cnn_uq90_preds[:,uq_num_val:,:]
cal_uq75_preds = cnn_uq75_preds[:,uq_num_val:,:]
cal_uq50_preds = cnn_uq50_preds[:,uq_num_val:,:]
cal_uq25_preds = cnn_uq25_preds[:,uq_num_val:,:]
cal_uq10_preds = cnn_uq10_preds[:,uq_num_val:,:]
cal_uq05_preds = cnn_uq05_preds[:,uq_num_val:,:]

uq_val_labels = uq_cal_labels[:uq_num_val,:]
val_uq95_preds = cnn_uq95_preds[:,:uq_num_val,:]
val_uq90_preds = cnn_uq90_preds[:,:uq_num_val,:]
val_uq75_preds = cnn_uq75_preds[:,:uq_num_val,:]
val_uq50_preds = cnn_uq50_preds[:,:uq_num_val,:]
val_uq25_preds = cnn_uq25_preds[:,:uq_num_val,:]
val_uq10_preds = cnn_uq10_preds[:,:uq_num_val,:]
val_uq05_preds = cnn_uq05_preds[:,:uq_num_val,:]


(2, 108559, 3)
(2, 5000, 3)


In [ ]:
# conformally adjust pinball estimates of ci
def get_adj_ci(pred, adj):
    return np.array((pred[0] - adj, pred[1] + adj))


In [196]:
# get quantile adjustment scalars for the three rate params
adj_cqr95 = uq.get_CQR(cal_uq95_preds, cal_uq_labels, inner_quantile=0.95)
adj_cqr90 = uq.get_CQR(cal_uq90_preds, cal_uq_labels, inner_quantile=0.90)
adj_cqr75 = uq.get_CQR(cal_uq75_preds, cal_uq_labels, inner_quantile=0.75)
adj_cqr50 = uq.get_CQR(cal_uq50_preds, cal_uq_labels, inner_quantile=0.50)
adj_cqr25 = uq.get_CQR(cal_uq25_preds, cal_uq_labels, inner_quantile=0.25)
adj_cqr10 = uq.get_CQR(cal_uq10_preds, cal_uq_labels, inner_quantile=0.10)
adj_cqr05 = uq.get_CQR(cal_uq05_preds, cal_uq_labels, inner_quantile=0.05)

# get validation quantiles
adj_val_uq = {}
adj_val_uq[0.05] = get_adj_ci(val_uq05_preds, adj_cqr05)
adj_val_uq[0.10] = get_adj_ci(val_uq10_preds, adj_cqr10)
adj_val_uq[0.25] = get_adj_ci(val_uq25_preds, adj_cqr25)
adj_val_uq[0.50] = get_adj_ci(val_uq50_preds, adj_cqr50)
adj_val_uq[0.75] = get_adj_ci(val_uq75_preds, adj_cqr75)
adj_val_uq[0.90] = get_adj_ci(val_uq90_preds, adj_cqr90)
adj_val_uq[0.95] = get_adj_ci(val_uq95_preds, adj_cqr95)


In [260]:
# get coverages and output files
val_coverage = uq.make_cqr_coverage_set(adj_val_uq, uq_val_labels[:,0:3])
make_output_files(val_coverage, adj_val_uq, "output/validation_CQR")

df_uq_val_labels = pd.DataFrame(uq_val_labels[:,0:3],
                             columns = ["R0", "delta", "m"])
df_uq_val_labels.to_csv("output/validation_CQR_labels.tsv", sep = "\t", index = False)

Quantile 0.05, parameter 0 finished: 4.44
Quantile 0.05, parameter 1 finished: 4.68
Quantile 0.05, parameter 2 finished: 4.56
Quantile 0.1, parameter 0 finished: 10.34
Quantile 0.1, parameter 1 finished: 10.68
Quantile 0.1, parameter 2 finished: 9.54
Quantile 0.25, parameter 0 finished: 25.76
Quantile 0.25, parameter 1 finished: 25.08
Quantile 0.25, parameter 2 finished: 25.1
Quantile 0.5, parameter 0 finished: 50.8
Quantile 0.5, parameter 1 finished: 50.08
Quantile 0.5, parameter 2 finished: 50.48
Quantile 0.75, parameter 0 finished: 75.48
Quantile 0.75, parameter 1 finished: 75.64
Quantile 0.75, parameter 2 finished: 74.86
Quantile 0.9, parameter 0 finished: 91.18
Quantile 0.9, parameter 1 finished: 89.9
Quantile 0.9, parameter 2 finished: 89.98
Quantile 0.95, parameter 0 finished: 95.42
Quantile 0.95, parameter 1 finished: 94.84
Quantile 0.95, parameter 2 finished: 95.32
ERROR! Session/line number was not unique in database. History logging moved to new session 3810


In [289]:
im.reload(uq)
test_lq, test_uq = uq.get_adaptive_CQR(cal_uq75_preds, cal_uq_labels, num_grid_points = 10, inner_quantile=0.75)

In [290]:
print(adj_val_uq[0.75][:,:,0])
(test_lq['R0'](val_uq75_preds[0][:,0]), test_uq['R0'](val_uq75_preds[1][:,0]))


[[4.2293116  6.27413291 4.50434221 ... 4.13068002 3.79668342 4.63846012]
 [4.61054962 6.8225571  5.84031886 ... 4.45836557 4.28433088 5.24517167]]


(array([4.20988418, 6.24898493, 4.48304618, ..., 4.11273994, 3.78377994,
        4.61643641]),
 array([4.63229266, 6.8422885 , 5.86791624, ..., 4.47928292, 4.30422878,
        5.27020281]))

In [ ]:


#####################################
## get coverages from experimente ###
#####################################



In [259]:
# function for computing all quantiles and populating an output dictionary
def get_cqr_ci(treeloc_tensor, prior_tensor):
    tm = train_means[0:3]
    tsd = train_sd[0:3]
    
    # PREDICT R0, sample rate, migration rate
    uq95 = q95_model.predict([treeloc_tensor, prior_tensor])
    uq90 = q90_model.predict([treeloc_tensor, prior_tensor])
    uq75 = q75_model.predict([treeloc_tensor, prior_tensor])
    uq50 = q50_model.predict([treeloc_tensor, prior_tensor])
    uq25 = q25_model.predict([treeloc_tensor, prior_tensor])
    uq10 = q10_model.predict([treeloc_tensor, prior_tensor])
    uq05 = q05_model.predict([treeloc_tensor, prior_tensor])
    
    # reversing normalization
    lin_uq95_preds = np.exp(cn.denormalize(uq95, tm, tsd))
    lin_uq90_preds = np.exp(cn.denormalize(uq90, tm, tsd))
    lin_uq75_preds = np.exp(cn.denormalize(uq75, tm, tsd))
    lin_uq50_preds = np.exp(cn.denormalize(uq50, tm, tsd))
    lin_uq25_preds = np.exp(cn.denormalize(uq25, tm, tsd))
    lin_uq10_preds = np.exp(cn.denormalize(uq10, tm, tsd))
    lin_uq05_preds = np.exp(cn.denormalize(uq05, tm, tsd))
    
    # adjust
    adj_uq = {}
    adj_uq[0.05] = get_adj_ci(lin_uq05_preds, adj_cqr05)
    adj_uq[0.10] = get_adj_ci(lin_uq10_preds, adj_cqr10)
    adj_uq[0.25] = get_adj_ci(lin_uq25_preds, adj_cqr25)
    adj_uq[0.50] = get_adj_ci(lin_uq50_preds, adj_cqr50)
    adj_uq[0.75] = get_adj_ci(lin_uq75_preds, adj_cqr75)
    adj_uq[0.90] = get_adj_ci(lin_uq90_preds, adj_cqr90)
    adj_uq[0.95] = get_adj_ci(lin_uq95_preds, adj_cqr95)
    
    return(adj_uq)
    

In [261]:
########################################
# compare APE of CNN to that of phylo ##
########################################

extant_data = pd.read_csv("../data_files/extant_phylocomp.cblv.csv", 
                   header =None, error_bad_lines = False, index_col = 0).to_numpy()

extant_labels = pd.read_csv("../data_files/extant_phylocomp_labels.csv",
                    header = None, error_bad_lines = False).to_numpy()




# compute and gather auxilliary prior data
extant_subsample_prop = extant_data[:,(max_tips-1) * 7]
phylocomp_mu = extant_data[:,(max_tips - 3) * 7]
extant_num_tips = cn.get_num_tips(extant_data)

aux_phylocomp = np.vstack((phylocomp_mu, extant_subsample_prop, extant_num_tips,
                          extant_labels[:,8], extant_labels[:,9])).transpose()

norm_aux_phylocomp = cn.normalize(aux_phylocomp, (train_aux_priors_means, train_aux_priors_sd))


# create input tensors
extant_treeLocation_tensor, extant_prior_tensor = cn.create_data_tensors(data = extant_data, 
                                                                                    mu = norm_aux_phylocomp[:,0],
                                                                                    subsample_prop = norm_aux_phylocomp[:,1],
                                                                                    num_tips = norm_aux_phylocomp[:,2],
                                                                                    tmrca = norm_aux_phylocomp[:,3],
                                                                                    mean_bl = norm_aux_phylocomp[:,4],
                                                                                    num_locs = num_locs,
                                                                                    max_tips = max_tips,
                                                                                    cblv_contains_mu_rho = True)



In [262]:
# compute CQR and make files
phylocomp_cqr = get_cqr_ci(extant_treeLocation_tensor, extant_prior_tensor)
phylocomp_coverage = uq.make_cqr_coverage_set(phylocomp_cqr, extant_labels[:,5:8])
make_output_files(phylocomp_coverage, phylocomp_cqr, "output/phylocomp_CQR")

Quantile 0.05, parameter 0 finished: 5.8
Quantile 0.05, parameter 1 finished: 6.52
Quantile 0.05, parameter 2 finished: 5.8
Quantile 0.1, parameter 0 finished: 10.14
Quantile 0.1, parameter 1 finished: 13.77
Quantile 0.1, parameter 2 finished: 11.59
Quantile 0.25, parameter 0 finished: 25.36
Quantile 0.25, parameter 1 finished: 23.19
Quantile 0.25, parameter 2 finished: 23.91
Quantile 0.5, parameter 0 finished: 58.7
Quantile 0.5, parameter 1 finished: 55.8
Quantile 0.5, parameter 2 finished: 52.17
Quantile 0.75, parameter 0 finished: 82.61
Quantile 0.75, parameter 1 finished: 81.16
Quantile 0.75, parameter 2 finished: 79.71
Quantile 0.9, parameter 0 finished: 94.2
Quantile 0.9, parameter 1 finished: 89.86
Quantile 0.9, parameter 2 finished: 89.13
Quantile 0.95, parameter 0 finished: 97.1
Quantile 0.95, parameter 1 finished: 94.2
Quantile 0.95, parameter 2 finished: 94.2
